In [29]:
import os
import random
import numpy as np
import pandas as pd
from tqdm import tqdm
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import random_split
from torch.utils.data import DataLoader, Dataset, Subset
from torch.utils.data import random_split, SubsetRandomSampler
from torchvision import datasets, transforms, models 
from torchvision.datasets import ImageFolder
from torchvision.transforms import ToTensor
from torchvision.utils import make_grid
from pytorch_lightning import LightningModule
from pytorch_lightning import Trainer
import pytorch_lightning as pl
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from PIL import Image

In [27]:
pip install torchvision

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [28]:
pip install pytorch-lightning


   ---------------------------------------- 0.0/801.6 kB ? eta -:--:--
   ---------------------------------------- 0.0/801.6 kB ? eta -:--:--
    --------------------------------------- 10.2/801.6 kB ? eta -:--:--
   ---- ----------------------------------- 92.2/801.6 kB 1.7 MB/s eta 0:00:01
   ----------- ---------------------------- 225.3/801.6 kB 2.0 MB/s eta 0:00:01
   --------------- ------------------------ 317.4/801.6 kB 2.2 MB/s eta 0:00:01
   ----------------- ---------------------- 348.2/801.6 kB 1.8 MB/s eta 0:00:01
   ------------------- -------------------- 399.4/801.6 kB 1.6 MB/s eta 0:00:01
   ------------------------ --------------- 481.3/801.6 kB 1.7 MB/s eta 0:00:01
   ---------------------------- ----------- 563.2/801.6 kB 1.6 MB/s eta 0:00:01
   ---------------------------- ----------- 563.2/801.6 kB 1.6 MB/s eta 0:00:01
   ---------------------------- ----------- 563.2/801.6 kB 1.6 MB/s eta 0:00:01
   ---------------------------- ----------- 563.2/801.6 kB 1.6 MB/s


[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
pip install numpy matplotlib seaborn scikit-learn tensorflow


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


Found 3750 images belonging to 8 classes.
Found 709 images belonging to 8 classes.
Found 710 images belonging to 8 classes.


NameError: name 'create_alexnet' is not defined

In [6]:
# Import necessary libraries
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import roc_curve, confusion_matrix
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models

# Define the function to create AlexNet model
def create_alexnet(input_shape, num_classes):
    model = models.Sequential()
    # Add layers to the model according to AlexNet architecture
    # Example:
    model.add(layers.Conv2D(96, (11, 11), strides=(4, 4), activation='relu', input_shape=input_shape))
    model.add(layers.MaxPooling2D(pool_size=(3, 3), strides=(2, 2)))
    # Add more layers...
    model.add(layers.Dense(num_classes, activation='softmax'))
    return model


In [30]:
transform=transforms.Compose([
        transforms.RandomRotation(10),      # rotate +/- 10 degrees
        transforms.RandomHorizontalFlip(),  # reverse 50% of images
        transforms.Resize(224),             # resize shortest side to 224 pixels
        transforms.CenterCrop(224),         # crop longest side to 224 pixels at center
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],
                             [0.229, 0.224, 0.225])
])

In [31]:
dataset0=datasets.ImageFolder(root="C:\\Users\\rocks\\Documents\\MAIN PROJECT\\MANGO LEAF DS",transform=None)
print(len(dataset0))
class_names=dataset0.classes
print(class_names)
print(len(class_names))

4000
['Anthracnose', 'Bacterial Canker', 'Cutting Weevil', 'Die Back', 'Gall Midge', 'Healthy', 'Powdery Mildew', 'Sooty Mould']
8


In [35]:
class DataModule(pl.LightningDataModule):
    
    def __init__(self, transform=transform, batch_size=32):
        super().__init__()
        self.root_dir = "C:\\Users\\rocks\\Documents\\MAIN PROJECT\\MANGO LEAF DS"
        self.transform = transform
        self.batch_size = batch_size

    def setup(self, stage=None):
        dataset = datasets.ImageFolder(root=self.root_dir, transform=self.transform)
        n_data = len(dataset)
        n_train = int(0.8 * n_data)
        n_test = n_data - n_train

        train_dataset, test_dataset = torch.utils.data.random_split(dataset, [n_train, n_test])

        self.train_dataset = DataLoader(train_dataset, batch_size=self.batch_size, shuffle=True)
        self.test_dataset = DataLoader(test_dataset, batch_size=self.batch_size)

    def train_dataloader(self):
        return self.train_dataset

    def test_dataloader(self):
        return self.test_dataset

In [36]:
class MyModel(LightningModule):

    def __init__(self):
        super().__init__()
        self.model_ = models.alexnet(pretrained=True)
        self.model_.classifier = nn.Sequential( nn.Linear(9216,1024),
                                             nn.ReLU(),
                                             nn.Dropout(p=0.5),
                                             nn.Linear(1024,len(class_names)),
                                             nn.LogSoftmax(dim=1))

    def forward(self, x):
        x = self.model_(x)
        return x

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=0.001)
        return optimizer

    def training_step(self, train_batch, batch_idx):
        X, y = train_batch
        y_hat = self(X)
        loss = F.cross_entropy(y_hat, y)
        pred = y_hat.argmax(dim=1, keepdim=True)
        acc = pred.eq(y.view_as(pred)).sum().item() / y.shape[0]
        self.log("train_loss", loss)
        self.log("train_acc", acc)
        return loss

    def validation_step(self, val_batch, batch_idx):
        X, y = val_batch
        y_hat = self(X)
        loss = F.cross_entropy(y_hat, y)
        pred = y_hat.argmax(dim=1, keepdim=True)
        acc = pred.eq(y.view_as(pred)).sum().item() / y.shape[0]
        self.log("val_loss", loss)
        self.log("val_acc", acc)

    def test_step(self, test_batch, batch_idx):
        X, y = test_batch
        y_hat = self(X)
        loss = F.cross_entropy(y_hat, y)
        pred = y_hat.argmax(dim=1, keepdim=True)
        acc = pred.eq(y.view_as(pred)).sum().item() / y.shape[0]
        self.log("test_loss", loss)
        self.log("test_acc", acc)


In [38]:
if __name__ == '__main__':
    datamodule = DataModule()
    datamodule.setup()
    model = MyModel()
    trainer = pl.Trainer(max_epochs=5)
    trainer.fit(model, datamodule)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs

  | Name   | Type    | Params
-----------------------------------
0 | model_ | AlexNet | 11.9 M
-----------------------------------
11.9 M    Trainable params
0         Non-trainable params
11.9 M    Total params
47.664    Total estimated model params size (MB)


Epoch 4: 100%|██████████| 100/100 [01:46<00:00,  0.94it/s, v_num=1]

`Trainer.fit` stopped: `max_epochs=5` reached.


Epoch 4: 100%|██████████| 100/100 [01:47<00:00,  0.93it/s, v_num=1]


In [41]:
   datamodule.setup(stage='test')
   test_loader = datamodule.test_dataloader()
   trainer.test(dataloaders=test_loader)

c:\Users\rocks\AppData\Local\Programs\Python\Python312\Lib\site-packages\pytorch_lightning\trainer\connectors\checkpoint_connector.py:145: `.test(ckpt_path=None)` was called without a model. The best model of the previous `fit` call will be used. You can pass `.test(ckpt_path='best')` to use the best model or `.test(ckpt_path='last')` to use the last model. If you pass a value, this warning will be silenced.
Restoring states from the checkpoint path at c:\Users\rocks\Documents\MAIN PROJECT\lightning_logs\version_1\checkpoints\epoch=4-step=500.ckpt
Loaded model weights from the checkpoint at c:\Users\rocks\Documents\MAIN PROJECT\lightning_logs\version_1\checkpoints\epoch=4-step=500.ckpt
c:\Users\rocks\AppData\Local\Programs\Python\Python312\Lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:441: The 'test_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=11` in the `DataLoa

Testing DataLoader 0: 100%|██████████| 25/25 [00:11<00:00,  2.09it/s]


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_acc          │    0.8324999809265137     │
│         test_loss         │    0.46235591173171997    │
└───────────────────────────┴───────────────────────────┘

[{'test_loss': 0.46235591173171997, 'test_acc': 0.8324999809265137}]